[Open with Colab](https://colab.research.google.com/github/1never/UEC_AIX_seminar2020/blob/master/elasticsearch_dialogue.ipynb)

In [ ]:
# Elasticsearchのダウンロードと解凍
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz

# Elasticsearchの日本語形態素解析用プラグイン analysis-kuromojiのインストール
!elasticsearch-7.0.0/bin/elasticsearch-plugin install analysis-kuromoji

# Pythonのelasticsearchライブラリのインストール
!pip install elasticsearch

In [8]:
# Elasticsearchの実行
!pkill -f elasticsearch
!chown -R daemon:daemon elasticsearch-7.0.0/bin/
!chown -R daemon:daemon elasticsearch-7.0.0/

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1))


In [ ]:
# 接続テスト (上記の実行から30秒ほど待つ必要があります)
!curl -X GET "localhost:9200/"

In [ ]:
# Pythonライブラリによる接続テスト
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch()
es.ping()

In [ ]:
# データのダウンロード
!git clone https://github.com/1never/UEC_AIX_seminar2020.git data

# SF作家 海野十三の小説から抽出した対話データを解凍
!unzip data/unno_pair.zip -y

# 青空文庫全データの対話データを解凍
!7za x data/aozora_pair.7z -y

In [ ]:
# 対話データをElasticsearchにインサート
# filename = './unno_pair.txt'
filename = './aozora_pair.txt'
def load():
    try:
      es.delete_by_query(index='dialogue_pair', body={"query": {"match_all": {}}})
      print("既存データを削除")
    except:
      print("削除対象データなし")
      pass

    with open(filename) as f:
        for i, __ in enumerate(f):
            print(i, '...', end='\r')
            __ = __.split('\t')
            query = __[0].strip()
            response = __[1].strip()
            item = {'_index':'dialogue_pair', '_type':'docs', '_source':{ 'query':query, 'response':response }}
            yield item

print(helpers.bulk(es, load()))

In [ ]:
# プログラムがエラーで落ちた場合，一時的にElasticsearchに接続できなくなりますが，一定時間経つことで接続可能になります．
es = Elasticsearch()

def get_reply(utterance, size=10):
  results = es.search(index='dialogue_pair', body={'query':{'match':{'query':utterance}}, 'size':size,})
  return [result['_source']['response'] for result in results['hits']['hits']]

res = None
logs = []
while(True):
  u = input("\n>")
  if "exit" == u:
    break
  elif u.isdecimal() and res is not None and int(u) < len(res):
    u = res[int(u)]
  elif "back" == u:
    logs.pop()
    u = logs.pop()


  res = get_reply(u)
  logs.append(u)
  for i, l in enumerate(logs):
    print("log " + str(i) + ": " , l)
  for i, r in enumerate(res):
    print(i, r)
 
# 使用方法
# 1. ">"の右の入力欄に最初のセリフを入力します
# 2. 現在までのログと検索された候補が表示されます．
# 3. 表示された候補の左の数字を">"の右の入力欄に入力することでその候補が次のセリフになります．数字以外のものを入力するとそれがセリフになります．
# 4. 入力を間違えた場合，「back」と入力すると前回の状態に戻ることができます．
# 5. 「exit」と入力すると終了します． 